In [2]:
import pandas as pd

# Lecture du fichier
#df = pd.read_excel("crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012.xlsx", sheet_name=None)
# Charger le fichier Excel
file_path = 'crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012.xlsx'
workbook = pd.ExcelFile(file_path)

def safe_int(val):
    try:
        return int(val)
    except (ValueError, TypeError):
        return None



In [3]:
import re  # Import manquant
import pandas as pd
# Relancer l'extraction avec l'import ajouté
flattened_data = []

# Filtrer les feuilles contenant "PN"
sheet_names = [sheet for sheet in workbook.sheet_names if "PN" in sheet]

for sheet in sheet_names:
    match = re.match(r"Services PN (\d{4})", sheet)
    if not match:
        continue
    year = int(match.group(1))
    
    df = workbook.parse(sheet, header=None)

    # Extraire les lignes clés
    peripheriques = df.iloc[1, 2:]  # Ligne 2, colonnes C -> fin
    libelles_index = df.iloc[2, 2:]  # Ligne 3, colonnes C -> fin
    titres_crimes = df.iloc[4:, 1]  # Colonne B à partir de ligne 5
    donnees = df.iloc[4:, 2:]  # Données à partir de ligne 5, colonnes C -> fin
    departements = df.iloc[0, 2:]  # Ligne 1, colonnes C -> fin

    # Parcourir les colonnes de données
    for col_idx, (departement_header, peripherique) in enumerate(zip(departements, peripheriques)):
        libelle = libelles_index.iloc[col_idx]
        departement = str(departement_header).strip()

        for row_idx, valeur in enumerate(donnees.iloc[:, col_idx]):
            flattened_data.append({
                "Année": year,
                "CGD/CSP": libelle,
                "Titre crime": titres_crimes.iloc[row_idx],
                "Département": departement,
                "Service": "Police",
                "Périmètre": peripherique,
                "Valeur": safe_int(valeur)
            })

# Créer le DataFrame final
df_flattened_pn = pd.DataFrame(flattened_data)

# Afficher les 10 premières lignes
df_flattened_pn.head(10)


,Année,CGD/CSP,Titre crime,Département,Service,Périmètre,Valeur
0,2012,DIDPAF PREVESSIN 01,Homicides pour voler et à l'occasion de vols,01,Police,DCPAF,0
1,2012,DIDPAF PREVESSIN 01,Homicides pour d'autres motifs,01,Police,DCPAF,0
2,2012,DIDPAF PREVESSIN 01,Tentatives d'homicides pour voler et à l'occas...,01,Police,DCPAF,0
3,2012,DIDPAF PREVESSIN 01,Tentatives homicides pour d'autres motifs,01,Police,DCPAF,0
4,2012,DIDPAF PREVESSIN 01,Coups et blessures volontaires suivis de mort,01,Police,DCPAF,0
5,2012,DIDPAF PREVESSIN 01,Autres coups et blessures volontaires criminel...,01,Police,DCPAF,0
6,2012,DIDPAF PREVESSIN 01,Prises d'otages à l'occasion de vols,01,Police,DCPAF,0
7,2012,DIDPAF PREVESSIN 01,Prises d'otages dans un autre but,01,Police,DCPAF,0
8,2012,DIDPAF PREVESSIN 01,Sequestrations,01,Police,DCPAF,0
9,2012,DIDPAF PREVESSIN 01,Menaces ou chantages pour extorsion de fonds,01,Police,DCPAF,0


In [4]:
df_flattened_pn.shape

(729386, 7)

In [24]:
# Filtrer les feuilles contenant "GN"
gn_sheet_names = [sheet for sheet in workbook.sheet_names if "GN" in sheet]

flattened_data_gn = []

for sheet in gn_sheet_names:
    match = re.match(r"Services GN (\d{4})", sheet)
    
    if not match:
        continue
    year = int(match.group(1))

    df = workbook.parse(sheet, header=None)

    # Récupération des métadonnées
    departements = df.iloc[0, 2:]  # Ligne 1 -> Département
    libelles_index = df.iloc[1, 2:]  # Ligne 2 -> Libellé index \ CGD
    titres_crimes = df.iloc[4:, 1]  # Colonne B à partir de ligne 5
    donnees = df.iloc[4:, 2:]  # Données à partir de ligne 5, colonnes C -> fin

    for col_idx, departement_header in enumerate(departements):
        libelle = libelles_index.iloc[col_idx]
        departement = str(departement_header).strip()

        for row_idx, valeur in enumerate(donnees.iloc[:, col_idx]):
            flattened_data_gn.append({
                "Année": year,
                "CGD/CSP": libelle,
                "Titre crime": titres_crimes.iloc[row_idx],
                "Département": departement,
                "Service": "Gendarmerie",
                "Périmètre": None,
                "Valeur": safe_int(valeur)
            })

# Créer le DataFrame final GN
df_flattened_gn = pd.DataFrame(flattened_data_gn)

# Afficher les 10 premières lignes
df_flattened_gn.head(10)


,Année,CGD/CSP,Titre crime,Département,Service,Périmètre,Valeur
0,2012,CGD BELLEY,Homicides pour d'autres motifs,01,Gendarmerie,None,0.0
1,2012,CGD BELLEY,Tentatives d'homicides pour voler et à l'occas...,01,Gendarmerie,None,0.0
2,2012,CGD BELLEY,Tentatives homicides pour d'autres motifs,01,Gendarmerie,None,3.0
3,2012,CGD BELLEY,Coups et blessures volontaires suivis de mort,01,Gendarmerie,None,0.0
4,2012,CGD BELLEY,Autres coups et blessures volontaires criminel...,01,Gendarmerie,None,227.0
5,2012,CGD BELLEY,Prises d'otages à l'occasion de vols,01,Gendarmerie,None,0.0
6,2012,CGD BELLEY,Prises d'otages dans un autre but,01,Gendarmerie,None,0.0
7,2012,CGD BELLEY,Sequestrations,01,Gendarmerie,None,0.0
8,2012,CGD BELLEY,Menaces ou chantages pour extorsion de fonds,01,Gendarmerie,None,7.0
9,2012,CGD BELLEY,Menaces ou chantages dans un autre but,01,Gendarmerie,None,54.0


In [25]:
# Concaténer les deux DataFrames PN et GN
df_final = pd.concat([df_flattened_pn, df_flattened_gn], ignore_index=True)

# Afficher les 10 premières lignes
df_final.head(10)

,Année,CGD/CSP,Titre crime,Département,Service,Périmètre,Valeur
0,2012,DIDPAF PREVESSIN 01,Homicides pour voler et à l'occasion de vols,01,Police,DCPAF,0.0
1,2012,DIDPAF PREVESSIN 01,Homicides pour d'autres motifs,01,Police,DCPAF,0.0
2,2012,DIDPAF PREVESSIN 01,Tentatives d'homicides pour voler et à l'occas...,01,Police,DCPAF,0.0
3,2012,DIDPAF PREVESSIN 01,Tentatives homicides pour d'autres motifs,01,Police,DCPAF,0.0
4,2012,DIDPAF PREVESSIN 01,Coups et blessures volontaires suivis de mort,01,Police,DCPAF,0.0
5,2012,DIDPAF PREVESSIN 01,Autres coups et blessures volontaires criminel...,01,Police,DCPAF,0.0
6,2012,DIDPAF PREVESSIN 01,Prises d'otages à l'occasion de vols,01,Police,DCPAF,0.0
7,2012,DIDPAF PREVESSIN 01,Prises d'otages dans un autre but,01,Police,DCPAF,0.0
8,2012,DIDPAF PREVESSIN 01,Sequestrations,01,Police,DCPAF,0.0
9,2012,DIDPAF PREVESSIN 01,Menaces ou chantages pour extorsion de fonds,01,Police,DCPAF,0.0


In [29]:

df_final["Service"] = df_final["Service"].str.strip().str.capitalize()
df_final["Périmètre"] = df_final["Périmètre"].fillna("Non spécifié")
print(df_final.shape,
df_final.Service.unique())



(1124606, 7) ['Police' 'Gendarmerie']


In [31]:
# Group by des données pour additionner les valeurs
df_final["Valeur"] = df_final["Valeur"].fillna(0).astype(int)
df_final = df_final.groupby(["CGD/CSP", "Service", "Titre crime", "Département", "Périmètre", "Année"]).agg({"Valeur": "sum"}).reset_index()
df_final.Service.unique()

array(['Police', 'Gendarmerie'], dtype=object)

In [32]:

df_final = df_final.groupby(["CGD/CSP", "Service", "Titre crime", "Département", "Périmètre", "Année"], as_index=False)["Valeur"].sum()
print(df_final.shape)

for date in df_final.Année.unique():
    print(date)
    # Enregistrer un dataframe par année
    df_final[df_final.Année == date].to_csv(f"crimes_{date}.csv", index=False)

(1092671, 7)
2020
2021
2018
2015
2016
2017
2019
2012
2013
2014
